# ABC Radio Recording

This notebook kicks off recording of an ABC radio stream, using `ffmpeg` on the embedded m3u8 stream from the webpage.

In [ ]:
# first, some environment setup (colab already has most of these installed)
! pip install beautifulsoup4 m3u8 requests
! apt install ffmpeg
# now, some actual python

In [ ]:
import bs4, json, m3u8, requests, subprocess
from time import gmtime, strftime

In [ ]:
# the ABC Radio webpage for the channel we want to record (ABC Hobart)
STATION_URL = 'https://www.abc.net.au/listen/live/hobart'

# the duration of the recording we want to capture, in seconds
DURATION_MINUTES = 30

In [ ]:
# use the station page to get the stream data
page_response = requests.get(STATION_URL)
page_response.raise_for_status() # throw an error if that didn't work

In [ ]:
# scrape this page for the stream data json
page_content = bs4.BeautifulSoup(page_response.content)
data = page_content.find('script', { 'id':'__NEXT_DATA__' }).text
data_json = json.loads(data)

In [ ]:
# now let's dive into this json to get the url for the m3u8 stream data
index_url = data_json["props"]["pageProps"]["data"]["componentsContent"][0]["componentProps"]["config"]["sources"][0]["file"]
# and then fetch from that url too
index_response = requests.get(index_url)
index_response.raise_for_status() # throw an error if that didn't work

In [ ]:
# now to decode it out of binary and then interpret as m3u8 playlist metadata
index_data = index_response.content.decode()
playlist_metadata = m3u8.loads(index_data)
# get the playlist therein with the highest bitrate
playlist = max(playlist_metadata.playlists, key=lambda x: x.stream_info.bandwidth)
playlist_uri = playlist.absolute_uri
playlist_uri

In [ ]:
# now to decide on a filename to write to
outfile = 'abchobart_' + strftime("%Y%m%d_%H%M%S", gmtime()) + '.wav'
# and construct the command to send to ffmpeg
command = f"ffmpeg -y -t {round(DURATION_MINUTES * 60.0)} -i {playlist_uri} {outfile}"
# and then run it
subprocess.run(command, shell=True)